In [ ]:
# Import required packages
from paik.solver import (
    Solver,
    DEFAULT_SOLVER_PARAM_M7,
    DEFAULT_SOLVER_PARAM_M3,
    DEFAULT_SOLVER_PARAM_M7_NORM,
)

num_poses = 100  # 100
num_sols = 1000  # 1000
solver = Solver(solver_param=DEFAULT_SOLVER_PARAM_M7_NORM)

# solver.latent = torch.zeros(1, 7).cuda()
solver.shrink_ratio = 0.25
solver.random_sample_solutions_with_evaluation(num_poses, num_sols, return_time=True)

In [ ]:
import numpy as np
import pandas as pd
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

P = solver._random_sample_poses(num_poses)  # type: ignore

# -> unrefined solutions
l2_errs = np.zeros((len(P), num_sols))
ang_errs = np.zeros((len(P), num_sols))

time_diffs = np.zeros((len(P)))

for i, p in enumerate(P):
    (
        solutions,
        l2_errors,
        angular_errors,
        joint_limits_exceeded,
        self_colliding,
        runtime,
    ) = ik_solver.solve(
        p, n=num_sols, refine_solutions=False, return_detailed=True
    )  # type: ignore

    solutions = solutions.detach().cpu().numpy()

    l2_errs[i] = l2_errors
    ang_errs[i] = angular_errors
    time_diffs[i] = runtime

l2_errs = l2_errs.flatten()

l2_errs.mean(), ang_errs.mean(), time_diffs.mean()